# Mixed SM 2021 Round 1

<table><tr>
<td><img src="https://raw.githubusercontent.com/elastufka/presentations/master/images/ZU_logo1.png" width="200"></td>
 <td><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSOgmNJ347YwD-sfSxjATPgVkPQh6XknjCv4Q&usqp=CAU" width='200'></td></tr>
    </table>
    
 **Some (incomplete) statistics, now with more information content**
 

## About the Plots

They are fully interactive! Just in case you have not encountered this before... it should be pretty easy to figure out when you hover your mouse in random places and it tells you what you can do.

I would not advise looking at this on a mobile phone.

## About the Data
**Skip if you're impatient**

- Statistics taken with UltiAnalytics app using GoPro footage. I couldn't always see the endzones. So if a thrower/receiver is marked as Anonymous with an Unknown gender matchup, that's where that comes from.

- Limitations of UltiAnalytics: All turns from our team are either drops or throwaways. So if a player got D'd it's counted as one of those (usually depending on how close to the camera it happened - if it was closer chances are higher I counted it as a drop because I could see what was going on)

- Timing data was useless here since I was often pausing or rewinding the video feed to try and figure out who was who 

- Data is anonymized since this is public (but probably hard to find) 

- Anonymized data and (messy) code available [here](https://github.com/elastufka/better_ultistats) . Feel free to check my work.

In [1]:
#py37
import pandas as pd 
import plotly
from plotly import graph_objects as go
import plotly.express as px
from ultistats_utils import *
from visualization_tools import *

In [3]:
#slideshow stuff
from IPython.display import Javascript
from plotly.offline import get_plotlyjs
Javascript(get_plotlyjs())
slidew=850
slideh=600
set_pride_template()

In [655]:
pcolors=pride_colors_plotly()
bcolors=pride_colors_plotly(flag='bi')
logo_layout=global_layout()

## Load & Prep Data

In [488]:
df=pd.read_csv('FucZH-stats.csv')

In [489]:
df.drop(columns=list(df.keys())[20:],inplace=True)

In [490]:
df['Passer_matchup']=['W' if p in wline else 'M' if p in mline else 'U' if p== 'Anonymous' else p for p in df.Passer]
df['Receiver_matchup']=['W' if p in wline else 'M' if p in mline else 'U' if p== 'Anonymous' else p for p in df.Receiver]
df['Defender_matchup']=['W' if p in wline else 'M' if p in mline else 'U' if p== 'Anonymous' else p for p in df.Defender]

In [675]:
break_type,hockey_assist,assist,scorer,gng,gngm,gngn,opp_ta=[],[],[],[],[],[],[],[]
turns,touches=[],[]
W4_M3=[]
M4_W3=[]

for idx,row in df.iterrows():
    #print(row,row['Line'])
    turns.append(is_turn(df,idx,row))
    touches.append(sort_touches(idx,row))
    seven_on=[row[k] for k in row.keys() if 'Player' in k]
    #print(seven_on)
    if gender_ratio(seven_on) == 1:
        W4_M3.append(1)
        M4_W3.append(0)
    else:
        W4_M3.append(0)
        M4_W3.append(1)
    #opponent throwaway?
    if row.Action=='Throwaway' and type(row.Passer)==float: #nan
        opp_ta.append(1)
    else:
        opp_ta.append(0)
    try:
        isgng,gm,gn=give_and_go(df,idx,row)
        if type(gn)==list:
            gn.sort()
            gn=tuple(gn)
        gng.append(isgng)
        gngm.append(gm)
        gngn.append(gn)
        assist.append(is_assist(df,idx,row))
        scorer.append(goal_scorer(df,idx,row))
        hockey_assist.append(is_hockey_assist(df,idx,row))
    except (KeyError,IndexError) as e:
        assist.append(None)
        scorer.append(None)
        hockey_assist.append(None)
        gng.append(None)
        gngm.append(None)
        gngn.append(None)
        pass
    if idx >0:
        break_type.append(is_break(row['Line'],row['Action'],row['Receiver']))
        #print(row['Line'],row['Action'],row['Receiver'],break_type)
    else:
        break_type.append(None)

In [670]:
#anonymize - might need to run this at the end just in case
for k,v in anony_dict.items():
    df.replace(k,v,inplace=True)

In [493]:
df['W4_M3']=W4_M3
df['M4_W3']=M4_W3
df['Offense_line']=[1 if l == 'O' else 0 for l in df.Line]

In [676]:
df['Break']=break_type
df['Hockey_assist']=hockey_assist
df['Scorer']=scorer
df['Assist']=assist
df['Turn']=turns
df['Touches']=touches
df['give-and-go']=gng
df['give-and-go_matchup']=gngm
df['give-and-go_pairs']=gngn
df['opponent_throwaway']=opp_ta

In [495]:
df['Pass_matchup']=df.Passer_matchup+df.Receiver_matchup
df['PointID']=[o+f"{op:02d}"+f"{tp:02d}" for o,op,tp in zip(df.Opponent,df['Our Score - End of Point'],df['Their Score - End of Point'])]

In [496]:
#dummies for 'Action' colump
adf=pd.get_dummies(df.Action)
#bdf=pd.get_dummies(df.Break)
#pmdf0=pd.get_dummies(df.Pass_matchup)
#pmdf0.rename(columns={'MW':'Pass_matchup_MW','WM':'Pass_matchup_WM','MM':'Pass_matchup_MM','WW':'Pass_matchup_WW','MU':'Pass_matchup_MU','UM':'Pass_matchup_UM','WU':'Pass_matchup_WU','UW':'Pass_matchup_UW'},inplace=True)
pmdf=pd.get_dummies(df.Touches)
pmdf.rename(columns={'U':'Touch_U','W':'Touch_W','M':'Touch_M'},inplace=True)
# rmdf=pd.get_dummies(df.Receiver_matchup)
# rmdf.rename(columns={'U':'Receiver_matchup_U','W':'Receiver_matchup_W','M':'Receiver_matchup_M'},inplace=True)
# dmdf=pd.get_dummies(df.Defender_matchup)
# dmdf.rename(columns={'U':'Defender_matchup_U','W':'Defender_matchup_W','M':'Defender_matchup_M'},inplace=True)
df=df.merge(adf,left_index=True,right_index=True)
#df=df.merge(bdf,left_index=True,right_index=True)
#df=df.merge(pmdf0,left_index=True,right_index=True)
df=df.merge(pmdf,left_index=True,right_index=True)
#df=df.merge(rmdf,left_index=True,right_index=True)
#df=df.merge(dmdf,left_index=True,right_index=True)
df.head(10)

,Date/Time,Tournamemnt,Opponent,Point Elapsed Seconds,Line,Our Score - End of Point,Their Score - End of Point,Event Type,Action,Passer,...,EndOfThirdQuarter,GameOver,Goal,Halftime,Pull,PullOb,Throwaway,Touch_M,Touch_U,Touch_W
0,2021-09-06 16:58,Mixed SM 2021,FSL,6,D,0,1,Defense,Pull,NaN,...,0,0,0,0,1,0,0,0,0,0
1,2021-09-06 16:58,Mixed SM 2021,FSL,6,D,0,1,Defense,Goal,NaN,...,0,0,1,0,0,0,0,0,0,0
2,2021-09-06 16:58,Mixed SM 2021,FSL,135,O,1,1,Offense,Catch,FS,...,0,0,0,0,0,0,0,1,0,0
3,2021-09-06 16:58,Mixed SM 2021,FSL,135,O,1,1,Offense,Catch,GA,...,0,0,0,0,0,0,0,1,0,0
4,2021-09-06 16:58,Mixed SM 2021,FSL,135,O,1,1,Offense,Catch,EL,...,0,0,0,0,0,0,0,0,0,1
5,2021-09-06 16:58,Mixed SM 2021,FSL,135,O,1,1,Offense,Catch,SH,...,0,0,0,0,0,0,0,1,0,0
6,2021-09-06 16:58,Mixed SM 2021,FSL,135,O,1,1,Offense,Goal,SB,...,0,0,1,0,0,0,0,0,0,1
7,2021-09-06 16:58,Mixed SM 2021,FSL,10,D,2,1,Defense,PullOb,NaN,...,0,0,0,0,0,1,0,0,0,0
8,2021-09-06 16:58,Mixed SM 2021,FSL,10,D,2,1,Defense,Throwaway,NaN,...,0,0,0,0,0,0,1,0,0,0
9,2021-09-06 16:58,Mixed SM 2021,FSL,10,D,2,1,Offense,Goal,JS,...,0,0,1,0,0,0,0,1,0,0


In [483]:
df.keys()

Index(['Date/Time', 'Tournamemnt', 'Opponent', 'Point Elapsed Seconds', 'Line',
       'Our Score - End of Point', 'Their Score - End of Point', 'Event Type',
       'Action', 'Passer', 'Receiver', 'Defender', 'Hang Time (secs)',
       'Player 0', 'Player 1', 'Player 2', 'Player 3', 'Player 4', 'Player 5',
       'Player 6', 'Passer_matchup', 'Receiver_matchup', 'Defender_matchup',
       'W4_M3', 'M4_W3', 'Offense_line', 'Break', 'Hockey_assist', 'Scorer',
       'Assist', 'Turn', 'Touches', 'give-and-go', 'give-and-go_matchup',
       'give-and-go_pairs', 'opponent_throwaway', 'Pass_matchup', 'PointID',
       'Catch', 'D', 'Drop', 'EndOfFirstQuarter', 'EndOfThirdQuarter',
       'GameOver', 'Goal', 'Halftime', 'Pull', 'PullOb', 'Throwaway',
       'Touch_M', 'Touch_U', 'Touch_W'],
      dtype='object')

In [687]:
df.to_csv('FucZH_SMR1_anonymized.csv')

### group by point

In [501]:
gdf=df.groupby('PointID')

## Plots - overall

In [662]:
exec(open('ultistats_utils.py').read())

## Game progression

Definitions:
- Hold = Start & Score on Offense
- Break = Start & Score on Defense

Good to Know: 
- Completed passes don't count goals because of the way UltiAnalytics categorizes actions and I was too lazy to put them back in

In [663]:
fig=game_flow_fig(df,'Fly High Lausanne',slidew,slideh)
fig.show()

In [664]:
fig=game_flow_fig(df,'Wombats Willisau',slidew,slideh)
fig.show()

In [665]:
fig=game_flow_fig(df,'Wizards Geneva',slidew,slideh)
fig.show()

In [666]:
fig=game_flow_fig(df,'FSL',slidew,slideh)
fig.show()

### Throwing and Receiving

In [17]:
#throws per offense, throws per defense
#fig=px.pie()
odf=df.where(df.Line=='O').where(df.opponent_throwaway == 0).dropna(how='all').groupby('Opponent')
ddf=df.where(df.Line=='D').where(df.opponent_throwaway == 0).dropna(how='all').groupby('Opponent')
tadf=df.where(df.opponent_throwaway==1).groupby('Opponent')

In [18]:
#defense_points=
#total_points=
offense_points=df.where(df.Offense_line==1).groupby('Opponent').PointID.nunique()
defense_points=df.where(df.Offense_line==0).groupby('Opponent').PointID.nunique()
total_points=offense_points+defense_points

In [671]:
fig=go.Figure()
fig.add_trace(go.Bar(x=df.Opponent.unique(),y=odf.Catch.sum()/offense_points,name='Offense catches'))
fig.add_trace(go.Bar(x=df.Opponent.unique(),y=ddf.Catch.sum()/defense_points,name='Defense catches'))
fig.add_trace(go.Bar(x=df.Opponent.unique(),y=odf.Throwaway.sum()/offense_points,name='Offense throwaway'))
fig.add_trace(go.Bar(x=df.Opponent.unique(),y=ddf.Throwaway.sum()/defense_points,name='Defense throwaway'))
fig.add_trace(go.Bar(x=df.Opponent.unique(),y=odf.Drop.sum()/offense_points,name='Offense drops'))
fig.add_trace(go.Bar(x=df.Opponent.unique(),y=ddf.Drop.sum()/defense_points,name='Defense drops'))
fig.add_trace(go.Bar(x=df.Opponent.unique(),y=odf.D.sum()/offense_points,name='Offense Ds'))
fig.add_trace(go.Bar(x=df.Opponent.unique(),y=ddf.D.sum()/defense_points,name='Defense Ds'))
fig.add_trace(go.Bar(x=df.Opponent.unique(),y=tadf.Throwaway.sum()/total_points,marker_color='grey',name='Opponent Throwaway'))
fig.update_layout(title='Average Events per Point')
fig.update_layout(logo_layout)

"Offense" and "Defense" here refer to line, not if we were currently playing O or D

In [657]:
ag=df.groupby('Hockey_assist').count().sort_values('Action',ascending=False)
aa=df.groupby('Assist').count().sort_values('Action',ascending=False)
fig=go.Figure()
fig.add_trace(go.Bar(x=ag.index,y=ag.Action.values,marker_color=pcolors[5],name='Hockey Assists'))
fig.add_trace(go.Bar(x=aa.index,y=aa.Action.values,marker_color=pcolors[6],name='Assists'))
fig.update_layout(barmode='stack',title='Assists and Hockey Assists',yaxis_title='Number',xaxis_title='Player',xaxis={'categoryorder':'total descending'})
fig.update_layout(logo_layout)

In [672]:
## pass matchups by gender
#ag=df.groupby('Pass_matchup').count().sort_values('Action',ascending=False)
fig=go.Figure()
for op in df.Opponent.unique():
    ag=df.where(df.Opponent==op).groupby('Pass_matchup').count().sort_values('Action',ascending=False)
    fig.add_trace(go.Bar(x=ag.index,y=ag.Action,name=op))
fig.update_layout(barmode='stack',title='Pass matchups',yaxis_title='Number of Passes',xaxis_title='Gender Matchup',xaxis={'categoryorder':'total descending'})
fig.update_layout(logo_layout)

Once again, here passes exclude goal-scoring passes.

If thrower/receiver is unknown, their gender matchup is also unknown (U)

In [673]:
fig=go.Figure()
for op in df.Opponent.unique():
    ag=df.where(df.Opponent==op).groupby('give-and-go_matchup').count().sort_values('Action',ascending=False)
    fig.add_trace(go.Bar(x=ag.index,y=ag.Action,name=op))
fig.update_layout(barmode='stack',title='Give-and-go matchups',yaxis_title='Number of Give-and-Go Passes',xaxis_title='Gender Matchup',xaxis={'categoryorder':'total descending'})
fig.update_layout(logo_layout)

A give-and-go was defined as a single instance where the thrower became the next receiver. So a sequence that looks like A-B-A-B-A is counted not as a single give-and-go, but as 3 (2 for A-B, 1 for B-A).

The order of the person in the give-and-go-pair does not matter for counting purposes.

In [677]:
ag=df.groupby('give-and-go_pairs').count().sort_values('Action',ascending=False).head(19)
xax=[p[0]+'-'+p[1] for p in ag.index.values]
colorscale = plotly.colors.sequential.Plasma
cdict=[[i/(len(colorscale)-1),c] for i,c in enumerate(colorscale)]
ccolors=[get_continuous_color(cdict,en/ag.Action.max()) for en in ag.Action.values]
fig=go.Figure()
fig.add_trace(go.Bar(x=xax,y=ag.Action.values,marker_color=ccolors))
fig.update_layout(title='Give-and-go Pairs',yaxis_title='Number of Give-and-Go Passes',xaxis_title='Pairs')#,xaxis={'categoryorder':'total descending'})
fig.update_layout(logo_layout)

In [520]:
#gender score - touch ratio / gender ratio
maj_men,maj_women=[],[]
for op in df.Opponent.unique():
    opdf=df.where(df.Opponent==op).dropna(how='all').groupby('PointID')
    tdf=opdf.agg({'W4_M3':np.median,'Touch_M':sum,'Touch_W':sum,'Touch_U':sum})
    aa=tdf.groupby('W4_M3')[['Touch_M','Touch_W','Touch_U']].mean()
    maj_men.append(aa.iloc[0].values)
    maj_women.append(aa.iloc[1].values)

In [565]:
maj_men=np.array(maj_men)
maj_women=np.array(maj_women)
maj_men.shape,maj_men[:,0]

((4, 3), array([2.        , 6.7       , 7.        , 3.18181818]))

In [686]:
#fig=go.Figure()
#for i,g in enumerate(['M','W']):
#    for j,op in enumerate(df.Opponent.unique()[:1]):
#        fig.add_trace(go.Bar(x=[-maj_women[j][i],maj_men[j][i]],name=op,hovertext=g))
#fig.show()
from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=maj_men[:,0],name='M touches (4M)',orientation='h',marker_color='green'),secondary_y=False)
fig.add_trace(go.Bar(x=maj_men[:,1],name='W touches (4M)',orientation='h',marker_color='orange'),secondary_y=False)
fig.add_vrect(x0=-6,x1=0,fillcolor='purple',opacity=.2)
fig.add_trace(go.Bar(x=-1*maj_women[:,0],name='M touches (4W)',orientation='h',marker_color='green'),secondary_y=True)
fig.add_trace(go.Bar(x=-1*maj_women[:,1],name='W touches (4W)',orientation='h',marker_color='orange'),secondary_y=True)
fig.add_trace(go.Scatter(x=maj_men[:,0]/maj_men[:,1],y=list(range(4)),mode='markers',marker_symbol='cross',marker_color='black',name='m/f touch ratio'),secondary_y=False)
fig.add_trace(go.Scatter(x=-maj_women[:,1]/maj_women[:,0],y=list(range(4)),mode='markers',marker_symbol='cross',marker_color='black',name='f/m touch ratio'),secondary_y=True)
fig.add_vline(x=4/3,line_dash='dash',line_width=.5)
fig.add_vline(x=-4/3,line_dash='dash',line_width=.5,name='4:3 ratio')
fig.update_yaxes(ticktext=[],tickvals=[])
fig.update_xaxes(ticktext=[4,2,0,2,4,6],tickvals=[-4,-2,0,2,4,6])
fig.update_yaxes(ticktext=df.Opponent.unique(),tickvals=[0,1,2,3],secondary_y=True)
fig.update_layout(title='Average Touches per Point',xaxis_title='Touches per Point')
fig.update_layout(logo_layout)

The dashed lines indicate the 4:3 ratio
    
Shaded area is where we played 4W 3M, unshaded is 4M 3W
